nitwit
======

## Challenge
I'm in my post-quantum crypto phase. Now featuring Winternitz one-time signatures.

```ncat --ssl nitwit.challs.pwnoh.io 1337```

## Included files
* nitwit.py

## Additional resources
The provided implementation cites the implementation as per [http://toc.cryptobook.us](http://toc.cryptobook.us). Version 0.6 is available from [https://crypto.stanford.edu/~dabo/cryptobook/BonehShoup_0_6.pdf](https://crypto.stanford.edu/~dabo/cryptobook/BonehShoup_0_6.pdf)

## Analysis
* $I_n^d$ is the set of vectors of size $n$ where each element $\in [0, d)$, hence $v \in I_n^d \to \left(v \in \mathbb{Z^d} \land \forall i \in \left[0,d\right) . (v_i \in \mathbb{Z} \land 0 \le v_i \lt n)\right)$. In Python, $I_n^d$ is represented as ```list[int]```
* $\mathcal{M}$ is the set of messages for signing. In Python, this is represented as ```bytes``

### Constants
* $v = 256$
* $\text{hash\_size} = 32$ (i.e. $v / 8$ - the number of bytes in $v$ bits)
* $d = 15$
* $n_0 = 64$
* $n_1 = \left\lfloor\log_{d + 1}(n_0)\right\rfloor + 1 = \left\lfloor\log_{16}(64)\right\rfloor + 1 = 2$
* $n = n_0 + n_1 = 66$

Additionally there is an assertion of:
$(d+1)^{n_0} = 2^{v}$

$16^{64} = 2^{256}$

${2^4}^{64} = 2^{256}$

$2^{4\times 64} = 2^{256}$

Which holds true

### Functions
#### ```get_hash(x: bytes) -> bytes``` 
SHA256 digest of x. Equivalent to $f(x)$

#### ```hash_chain(x: bytes, d:int) -> bytes```
Repeated application of get_hash, equivalent to $f^{(d)}(x)$

#### ```int_to_vec(m: int, vec_len: int, base: int) -> list[int]```
Converts $m$ to a vector of ints in base $base$ i.e. $I_{\text{base}}^{\text{vec\_len}}$

Partitions $m \in {0,1}^v$ into consecutive blocks of $log_2(d+1)$ bits

#### ```domination_free_function(m: int) -> list[int]```
Equivalent to $P : \mathcal{M} \to I_n^d $

Let $s$ be the representation of $m$ in $I_{n_0}^d$

Let $c = d * n_0 - (s_1 + ... + s_{n_0})$ = $960 - (s_1 + ... + s_{n_0}), c \in \left[0, dn_0\right] = [0,960]$

Let $c'$ be the representation of $c$ in $I_{n_1}^d$

Return the concatenation of $s$ and $c'$, hence $m' = \left(s_0, ... s_{n_0}, c'_0, ..., c'_{n_1}\right)$

#### ```Winternitz.__init__(self)```
Sets up the signing session:
* Defines a secret (a vector of $v=256$ random bytes)
* Seeds a random number generator with that secret. The documentation for random.Random explicity states that this should not be used as a secure random generator.
* Defines $xs = x_0..x_n$ where $x_i = $ a vector of $\text{hash\_size}$ random bytes
* Defines $ys = y_0..y_n$ where $y_i = f^{(d)}(x_n)$

#### ```Winternitz.public_key(self)```
Returns $f(y_0 | .. | y_n)$, that is the SHA256 hash of the concatenation (into a single byte array) of $ys$

#### ```Winternitz.sign(self, m:bytes)```
Asserts $8 \times \text{len}(m) <= v$ i.e. m is up to $\text{hash\_size} = 32$ bytes

Computes the domination free function $ss = P(m \text{ as big endian int})$

Returns $\sigma = \left(f^{(s_1)}(x_1), ..., f^{(s_n)}(x_n)\right)$ 

#### ```Winternitz.verify(self, public_key: bytes, m: bytes, signature: list[bytes])```
Computes the same domination free function $ss - P(m \text{ as big endian int})$

Takes the current signature, and completes the hash chaining, by hashing each section of the signature by the remaining $d - s$ times, to reveal the public key.

## Attack
Let's play around with some values in the function:

In [1]:
from nitwit import *

w = Winternitz()
pk = w.public_key()
print(f"Public key:", pk.hex())

ss = domination_free_function(int.from_bytes(b"1234", "big"))
print(f"domination_free_function:", ss)
print(len(ss))

sig = w.sign(b"")
print(f"Signature:")
print("\n".join([s.hex() for s in sig]))

print("\nPrivate key:")
print("\n".join([s.hex() for s in w.xs]))

Public key: 2113bd4e6b951029f98d85194e472e3d5f0e65cf4de10465b982f1b85872fc89
domination_free_function: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 3, 2, 3, 3, 3, 4, 10, 3]
66
Signature:
267054a6d655c8191e24fc45b182a45a18c7b3d9f23d0b278f5c4d4b9ea053c0
4c62cdf756a91f1906d5198a3a7c3e7ae6a9ed2302dd934c1654cd8869d6c93f
1260f255ec140ea40c489496f8c32bd974e1ed0f011293831b7fa8afee88b26e
b88af15763859a871a4825cf5515dc6eedb64d76cc3ff1aaa2adf14875c382fc
0f4f74263979401bfe33d149a61cde52f24313dfcd21cdc4b905616913850f69
ea441a2a2d19938790e19431da964c6ff5a1090bbd6f233d2e1671b8a0c65c13
c42d5d1c0529547ef84fb6b56cd36a72d0d70d8396aa9c10121a154699541ac6
81f15611e4794277641705bfe8cdece1d57b93461dfc3812e7febaba716dadb0
931edee07de04bf46a944eaae456285889f029bd8e17fd371251cdd28aa68c4c
a162ad5f67601c34ca094a8a0d41b184abbaa09989c7c50a9d5fa580e4f706a9
5a5bd41acd5d04e3e28f0640f2ec53915

It seems to me that if we can generate two inputs, such that the each value from the domination free function is the same or equal to the previous, we can then hash the corresponding sections of the signature to make up for the shortfall, generating a valid signature.

Let's include the phrase "Admin" in the message as well, so that the final message only requires that single bitflip from 0x41 to 0x61

In [2]:
authMessages = []

def be(message):
    return int.from_bytes(message, "big")

def all_elements_greater(xs, ys):
    for x, y in zip(xs, ys):
        if x < y:
            return False
    return True

def dom(message):
    m_vec = int_to_vec(message, n_0, d+1)
    
    c = (15 * 64) - sum(m_vec)
    c_vec = int_to_vec(c, n_1, d + 1)

    return m_vec, c_vec

def generate(message, ms = None):
    m_vec, c_vec = dom(be(message))
    ss = m_vec + c_vec
    if ms:
        ms.append((message, ss))
    print(''.join(["_" if s == 0 else f"{s:x}" for s in m_vec]) + " / " + ' '.join(["" if s == 0 else f"{s:x}" for s in c_vec]))

    return (ss, ms)

generate(b"Admin", authMessages);
generate(b"admin", authMessages);

print(domination_free_function(be(b"Admin")))
print(domination_free_function(be(b"admin")))

______________________________________________________41646d696e / 7 3
______________________________________________________61646d696e / 7 3
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 1, 6, 4, 6, 13, 6, 9, 6, 14, 7, 3]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 1, 6, 4, 6, 13, 6, 9, 6, 14, 7, 3]


After some fiddling, I found that Admin and admin have the same checksum, but only the one signature varies. Let's now test the hypothesis using a stripped down version of main

In [3]:
w = Winternitz()
pk = w.public_key()
print(f"Public key: {pk.hex()}")
m = bytes.fromhex(b"Admin".hex())
if b"admin" in m:
    raise "Not authorized"
    

sig = w.sign(m)
print(f"Your signature is:")
print(sig)

m_new = bytes.fromhex(b"admin".hex())
if m == m_new:
    raise "Repeated message"

## forge signature
old_dom = domination_free_function(int.from_bytes(m, "big"))
new_dom = domination_free_function(int.from_bytes(m_new, "big"))
forged_sig = sig.copy()

for i in range(len(new_dom)):
    if (new_dom[i] < old_dom[i]):
        raise "can't hash backwards"
    forged_sig[i] = hash_chain(forged_sig[i], new_dom[i] - old_dom[i])

print(forged_sig)
if type(forged_sig) is not list:
    raise ("Bad signature")
if len(forged_sig) != n:
    raise("Bad signature")
if not all(type(x) is bytes for x in forged_sig):
    raise("Bad signature")
if not all(len(x) == hash_size for x in forged_sig):
    raise("Bad signature")

if w.verify(pk, m_new, forged_sig):
    if b"admin" in m_new:
        print("You must be the admin, so here's your flag:")
        print(flag)
    else:
        print("Valid signature, but you're not admin")
else:
    print("Signature failed to verify")

Public key: bc9dc87762255a043ebcaab6112fa5d2def19a2dd80b7eb0ebe2d75eda4001e4
Your signature is:
[b'/p\xd1"\xd8m\x05,\x0c\x9a7\xbbi\xd9\xe6\x9d\xd6r\x97\x14\xbb\xca2\xb6\xb6\x88]\xf5\xe4\xc8\xcbr', b'&q\r\x0f\x10\xc6\xef\x0f\xc4T:\xb3,\x1b\n\xd0\x11$p\x92n\xae\xeb\x8f\xbc\x8c\x8am\xf1\xb69\xd0', b'>\xf7\x859\xed\xd3\x94`\x14\xae\n]\xae\xa3^\x1d\xa8\xa6I\xc5nR\xff\xc8\xc9\x19;\x98\xa2\xf4\x1c\x86', b'!\xe3\x843\xc5B\xdd\x10\x90\xab\x1d9[\xf6+T\xbfU1q]\x13O\xad\x1a\x98E\x9a\x04\xde-\xda', b'\x90\xf8\xf9R\xce\xa6\x9dL\xbc\xf5H*\xb5\x86\x9e\xde8%\x80\xcf\xe3\xc0\x00LY\xd8i\xda\x83\xd7\xd9\xa4', b"\tO\x0c\x11\xa5\n\xf15e\xc3/\xe6\xa7\x95\xfc\xd2\x8a\xb7.\x80\x98\xb3I,<\xbc\xcb'\x0e\xa3\xa9K", b'\xfb\x0c\x80\x03&\x92?\x00\x97\x19\xba\xfc\x12\xd6.\xec\rO\xd7C\xc3\xfeok)\xd0\xdd\xa9+\xa39\xf8', b'`\xfa\xb4\x1e\x10L8\x13\x1ag\x98X\xa1I\xc5\xd5\x18=\\\xde\xf2\xf0\xba\xb5\xdc\xbfo\xdb\x19\xa5\x87\xbf', b'X\x0f\xf3\xa7K\x99\x80Jk\x85f\xa0X\xf3\n\xad$TC\xb9\x97\xcc\xb6b\xd0\xc0V"\xf9\xdbM\x1b', b'cm

And now let's test against the server

In [4]:
from pwn import *

# context.log_level = "debug"
conn = remote("nitwit.challs.pwnoh.io", 1337, ssl=True)

conn.readuntil(b">>> ")
conn.sendline(b"Admin".hex())
conn.recvline()

sig = ast.literal_eval(conn.recvlineS())

old_dom = domination_free_function(int.from_bytes(m, "big"))
new_dom = domination_free_function(int.from_bytes(m_new, "big"))
forged_sig = sig.copy()

for i in range(len(new_dom)):
    if (new_dom[i] < old_dom[i]):
        raise "can't hash backwards"
    forged_sig[i] = hash_chain(forged_sig[i], new_dom[i] - old_dom[i])

conn.sendline(b"admin".hex())
conn.sendline(str(forged_sig))
print(conn.recvallS())

[x] Opening connection to nitwit.challs.pwnoh.io on port 1337
[x] Opening connection to nitwit.challs.pwnoh.io on port 1337: Trying 2600:1f16:75:1c01::4
[+] Opening connection to nitwit.challs.pwnoh.io on port 1337: Done


/tmp/ipykernel_327706/1223502806.py:7: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  conn.sendline(b"Admin".hex())


[x] Receiving all data
[x] Receiving all data: 93B
[x] Receiving all data: 97B


/tmp/ipykernel_327706/1223502806.py:21: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  conn.sendline(b"admin".hex())
/tmp/ipykernel_327706/1223502806.py:22: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  conn.sendline(str(forged_sig))


[x] Receiving all data: 118B
[x] Receiving all data: 4.12KB
[x] Receiving all data: 6.27KB
[x] Receiving all data: 6.35KB
[+] Receiving all data: Done (6.35KB)
[*] Closed connection to nitwit.challs.pwnoh.io port 1337

Can you forge a signature for another message?
Enter a new message to sign as a hex string:
>>> Enter signature:
>>> [b'\xe7\xea\x9a\x86V1\xe9w\x86~\xb1\xe4\xf0\x94\xbb\xaf\r=yC\x9e\xd0\t\xf1{\x9e+3\xbd#\xc8\xe6', b'\xea\xc7\x07\xac\xbe\xa4(\x15\x97\x88WJ\xc9\x1d\x92\xbf\xe41.\xcf`\x10K\xc8\xd4\xeb\xd8\xee[~r\xad', b'\x95\x11m\x18\xeb\xd2Yj\xc8a\x0c\xcc@[1\x8fx\x88\xc4\xba\xa1W\x80\t\x8a`B\x88\xa0\x84\xe7\x1a', b'\xd0\x10\r\xdb\xe8E\xfe\xf8\xf7\x0c\\\xf8\x83KD\xe4>(\xae\xd9um\x1dkK\xcdt_\xda\x8dg\xef', b'\xc5\xb8\xaapu1\xe8u\xabz\x93\xc4\xdaX\xf5nD>\\\xe7\xeb68,\xb6\xd5#\x0c\xe4\xd3\x98j', b']\x81\xdd\xb3\x0cR\x11M\x86\xcf+\xb1_\xd4\xdb\x08\x9e\x17L\xe6\x04L\x82\xdc\x11z\xd6\xb7D\x8f\x93<', b'\x13w\xdf\xf67\xaf\xca:?\xbe\x1cRr\x9f9\\\xccu\xf0\xa5\x8a\xe6\x80-\xcc\xd2.!N\